__purpose__ : get some numbers for tables and supplement

In [1]:
from pythonimports import *
latest_commit()

##################################################################
Current commit of pythonimports:
commit b1d8bd7312fbf3c6afef4ad9ea2585831ec509a5
Author: Brandon Lind <lindb@vcu.edu>
Date:   Fri Feb 12 12:21:51 2021 -0500
Today:	March 10, 2021 - 11:29:52
python version: 3.8.5
##################################################################



# determine megaSNP intersection with baseline-filtered SNPs

#### load baseline-filtered SNPs

In [2]:
# code from 001_testdata_explore.ipynb
# get the file names
sppfiles = {'gatk':{'JP': '/data/projects/pool_seq/non-pangenome/gatk_diploid_testdata/JP_i101-gatk/filtered_snps/JP_i101_filtered_concatenated_snps_max-missing_table_biallelic-only_translated.txt',
                   'DF': '/data/projects/pool_seq/non-pangenome/gatk_diploid_testdata/DF_i52-gatk/filtered_snps/DF_i52_filtered_concatenated_snps_max-missing_table_biallelic-only_p000_translated.txt'},
           'varscan':{'JP': '/data/projects/pool_seq/non-pangenome/varscan_pooled/JP_pooled/snpsANDindels/JP_pooled-varscan_all_bedfiles_SNP_translated.txt',
                      'DF': '/data/projects/pool_seq/non-pangenome/varscan_pooled/DF_p52/snpsANDindels/DF_p52-varscan_all_bedfiles_SNP.txt'}}
# read in the tables
tablecols = ['CHROM','POS','REF','ALT','AF','QUAL','TYPE','FILTER','ADP','WT','HET','HOM','NC']
snps = {}
for method in sppfiles:
    print(method)
    snps[method] = {}
    for spp in sppfiles[method]:
        print('\t',spp)
        snps[method][spp] = pd.read_table(sppfiles[method][spp])
        if method == 'varscan' and spp == 'JP':
            # i ran all pools, isolate JP_p101 calls
            locuscols = [col for col in snps[method][spp] if 'locus' in col]
            snps[method][spp] = snps[method][spp][tablecols + ['JP_p101.GT','JP_p101.GQ','JP_p101.SDP',
                                                               'JP_p101.DP','JP_p101.FREQ','JP_p101.PVAL'] +locuscols]
            # remove AF=0 (because SNPs were called across all pools, JP_p101 can have AF=0)
            # AF=0 wouldn't be possible if calling AFs on a single pool
            snps[method][spp] = snps[method][spp][snps[method][spp]['JP_p101.FREQ'] != "0%"]
            # remove rows with no calls
            snps[method][spp] = snps[method][spp][snps[method][spp]['JP_p101.FREQ'] == snps[method][spp]['JP_p101.FREQ']]
            # fill in AF column
            snps[method][spp]['AF'] = snps[method][spp]['JP_p101.FREQ'].str.replace("%", "").astype(float)/100
        elif method =='varscan' and spp == 'DF':
            df = snps[method][spp]  # view, will changes go to snps['varscan']['DF']
            # add in a ">" so locus names match between gatk and varscan
            df['CHROM'] = [f">{chrom}" for chrom in df['CHROM'].tolist()]
            df['locus'] = [f">{locus}" for locus in df['locus'].tolist()]
# note varscan will output AF=0 (fixed for REF), I'm leaving these in so we can compare AFs of these too

gatk
	 JP
	 DF
varscan
	 JP
	 DF


In [3]:
# remove noREF SNPs
for method,spp in snps.items():
    for sp,df in spp.items():
        if method == 'gatk':
            snps[method][sp] = df[~df['ALT'].str.contains('+', regex=False)]
        print(method, sp, nrow(snps[method][sp]))
        
# original submission
# gatk JP 377080
# gatk DF 1526554
# varscan JP 3686528
# varscan DF 1601285

gatk JP 377080
gatk DF 1526554
varscan JP 3686528
varscan DF 1601285


In [4]:
# code from 003_testdata_validate_megaSNPs.ipynb
# load megaSNPs
megafiles = {'JP': '/data/projects/pool_seq/non-pangenome/varscan_mega/JP_RFmg7/snpsANDindels/JP_RFmg7-varscan_all_bedfiles_SNP_paralog_snps_translated.txt',
             'DF': '/data/projects/pool_seq/non-pangenome/varscan_mega/DF_megaSNPs_from_download/DF_mega/snpsANDindels/02_baseline_filtered/DF_mega-varscan_all_bedfiles_SNP_paralog_snps.txt'}
megasnps = {}
for sp,file in megafiles.items():
    megasnps[sp] = pd.read_table(file)
    locuscol = 'unstitched_locus' if 'unstitched_locus' in megasnps[sp].columns else 'locus'
    if locuscol == 'locus':
        loci = [">%s" % locus for locus in megasnps[sp][locuscol].tolist()]
    else:
        loci = megasnps[sp][locuscol].tolist()
    megasnps[sp][locuscol] = loci
    megasnps[sp].index = loci
    print(sp, nrow(megasnps[sp]))

# original submission
# JP 32751
# DF 398774

JP 32751
DF 398774


In [5]:
# get intersection
for method,spp in snps.items():
    for sp,df in spp.items():
        snpslocuscol = 'unstitched_locus' if 'unstitched_locus' in df.columns else 'locus'
        megalocuscol = 'unstitched_locus' if 'unstitched_locus' in megasnps[sp].columns else 'locus'
        inter = set(df[snpslocuscol]).intersection(megasnps[sp][megalocuscol])
        print(method, sp, len(inter))
# # original submission
# gatk JP 7408
# gatk DF 293
# varscan JP 25500
# varscan DF 825

gatk JP 7408
gatk DF 293
varscan JP 25500
varscan DF 825


# load trimming and mapping rates

In [6]:
def evaluate_readinfo(df):
    """Print out some stats from the readinfo.txt file."""
    # q30
    print(ColorText('\n\tBase Quality').bold())
    totalbefore = np.nansum(df['total_bases-before_trimming'])
    totalafter = np.nansum(df['total_bases-after_trimming'])
    q30before = np.nansum(df['q30_bases-before_trimming'])
    q30after = np.nansum(df['q30_bases-after_trimming'])
    percbefore = '\t\tperc before = %s%%' % (round((q30before/totalbefore)*100,2))
    percafter = '\t\tperc after = %s%%' % (round((q30after/totalafter)*100,2))
    print(percbefore)
    print(percafter)

    # trimming extent
    print(ColorText('\n\tTrimming').bold())
    before = np.nansum(df['total_reads-before_trimming'])
    after = np.nansum(df['total_reads-after_trimming'])
    print('\t\tbefore = %s' % before)
    perc = round(after/before, 4)
    print('\t\tperc = %s%%' % (round((after/before)*100,2)))

    # mapping
    print(ColorText('\n\tMapping').bold())
    try:
        mapped = np.nansum(df['mapped_bamfile'])
        percmapped = '\t\tperc mapped = %s%%' % round((mapped/after)*100, 2)
        rmdup = np.nansum(df['dedup_bamfile'])
        percnodups = '\t\tperc of mapped remaining after removing duplicates = %s%%' % round((rmdup/mapped)*100,2)
        print(percmapped)
        print(percnodups)
    except:
        pass
    

#### read in the files, print out data

In [7]:
# in some instances, the table I'm loading has other info I don't need here, use convert to reduce df
convert = {'gatk JP': 'JP_101',
           'gatk DF': 'DF_52',
           'varscan JP': 'JP_p101',
           'varscan DF': 'DF_p52',
           'varscan mega JP': 'JP_RFmg7'}
files = ['/data/projects/pool_seq/non-pangenome/gatk_diploid_testdata/DF_i52-gatk/readinfo.txt',
         '/data/projects/pool_seq/non-pangenome/gatk_diploid_testdata/JP_i101-gatk/readinfo.txt',
         '/data/projects/pool_seq/non-pangenome/varscan_pooled/DF_p52/readinfo.txt',
         '/data/projects/pool_seq/non-pangenome/varscan_pooled/JP_pooled/readinfo.txt',
         '/data/projects/pool_seq/non-pangenome/varscan_mega/DF_megaSNPs_from_download/DF_mega/readinfo.txt',
         '/data/projects/pool_seq/non-pangenome/varscan_mega/JP_RFmg7/readinfo.txt']
# read in the files, reduce, and print sequencing statistics
dfs = {}
for f in files:
    df = pd.read_table(f)
    print(nrow(df))
    method = 'gatk' if 'gatk' in f else 'varscan'
    if 'mega' in f:
        method = method + ' mega'
    sp = 'JP' if 'JP_' in f else 'DF'
    msp = method + ' ' + sp
    if msp in convert:
        df = df[df['samp'].str.contains(convert[msp], regex=False)]
    dfs[msp] = df
    print(nrow(df))
    print(ColorText(f'\n{method} {sp}').bold().blue())
    evaluate_readinfo(df)
#     display(df)

40
40

gatk DF

	Base Quality
		perc before = 88.92%
		perc after = 90.94%

	Trimming
		before = 419786724.0
		perc = 98.07%

	Mapping
		perc mapped = 85.48%
		perc of mapped remaining after removing duplicates = 68.68%
44
40

gatk JP

	Base Quality
		perc before = 87.28%
		perc after = 89.92%

	Trimming
		before = 392241534.0
		perc = 97.4%

	Mapping
		perc mapped = 46.14%
		perc of mapped remaining after removing duplicates = 79.4%
42
2

varscan DF

	Base Quality
		perc before = 88.76%
		perc after = 90.44%

	Trimming
		before = 109897366.0
		perc = 99.05%

	Mapping
		perc mapped = 86.31%
		perc of mapped remaining after removing duplicates = 41.55%
118
2

varscan JP

	Base Quality
		perc before = 86.55%
		perc after = 89.04%

	Trimming
		before = 152988120.0
		perc = 98.06%

	Mapping
7
7

varscan mega DF

	Base Quality
		perc before = 95.63%
		perc after = 97.55%

	Trimming
		before = 203296026.0
		perc = 97.16%

	Mapping
		perc mapped = 83.52%
		perc of mapped remaining after remov